In [1]:
import sys
sys.path.append('./code')

import matrix_elements as me
import numpy as np
import copy

In [2]:
class HundsCaseABasisState(object):
    
    def __init__(self,T=None, j=None, m=None, w=None):
        if j is None or m is None or w is None:
            raise IOError('j, m, and w must be defined.')
        self.j = j
        self.m = m
        self.w = w
        self.T = T
    
    def __str__(self):
        output =  ( 'j=' + str(self.j) + ', '
                  + 'm=' + str(self.m) + ', '
                  + 'w=' + str(self.w))
        
        if self.T is not None:
            output = (str(self.T) + ', ' + output)
        
        output = '|' + output + '>'
        return output
        
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

    
def hunds_case_a_basis(T=None, j=None, m=None, w=None):
    
    states = []
    
    if j is None or w is None:
        raise IOError('j and w must be defined')
    if not(isinstance(j,(list, tuple))):
        j = [j]
    if not(isinstance(w,(list, tuple))):
        w = [w]
    if not(isinstance(T,(list, tuple))):
        T = [T]
    if m is not None and not(isinstance(m,(list, tuple))):
        m = [m]
        
    for T_inst in T:
        for j_inst in j:
            for w_inst in w:
                if m is None:
                    m_temp = range(-j_inst, j_inst + 1)
                else:
                    m_temp = m
                    
                for m_inst in m_temp:
                    
                    basis_state = HundsCaseABasisState(T=T_inst, j=j_inst,
                                                       m=m_inst, w=w_inst)
                    state = State(1,basis_state)
                    states.append(state)
                    
                    
    if len(states) == 1:
        states = states[0]
    
    return states


class State(object):
    
    def __init__(self, amplitudes, states):
        
        if not(isinstance(amplitudes,(list,tuple)) 
               or amplitudes.__class__.__name__ == 'ndarray'):
            amplitudes = [amplitudes]
            
        if not(isinstance(states, (list,tuple))):
            states     = [states]
            
        if len(amplitudes) != len(states):
            raise IOError("""the state object must be provided equal
                          length amplitude and states lists""")
            
        self.amplitudes = np.array(amplitudes)
        self.states     = list(states)
        
    def __len__(self):
        return len(self.states)
        
    def __str__(self):
        output = []
        for a, s in zip(self.amplitudes, self.states):
            output.append(' + ')
            if a != 1:
                output.append(str(a))
            output.append(str(s))
            
        output.pop(0)
        output = ''.join(output)
        return output
    
    def normalize(self):
        self.amplitudes = self.amplitudes / self.norm()
        
    def norm(self):
        return np.sqrt(np.sum(self.amplitudes * np.conj(self.amplitudes)))
    
    def t(self):
        return State(np.conj(self.amplitudes),self.states)
    
    def __repr__(self):
        return str(self)
    
    def __mul__(self, other):
        if isinstance(other, self.__class__):
            if other.states == self.states:
                return sum(np.conj(other.amplitudes) * self.amplitudes)
            
            if len(self.states) >= len(other.states):
                large = self.t()
                small = other
            else:
                large = other
                small = self.t()
                
            total = 0
            for s, a in zip(small.states, list(small.amplitudes)):
                if s in large.states:
                    ind = large.states.index(s)
                    total += a * large.amplitudes[ind]  
            return total
        
        else:
            return State(self.amplitudes * other, self.states)
    
    def __rmul__(self, other):
        if isinstance(other, self.__class__):
            return np.conj(self.__mul__(other))
        else:
            return self.__mul__(other)
    
    def __add__(self, other):
        if not(isinstance(other, self.__class__)):
            raise TypeError('only state objects can be added together')
            
        new_states = copy.deepcopy(self.states)
        new_amplitudes = list(copy.deepcopy(self.amplitudes))
        
        for s, a in zip(other.states,list(other.amplitudes)):
            if s in new_states:
                ind = new_states.index(s)
                new_amplitudes[ind] += a
            else:
                new_states.append(s)
                new_amplitudes.append(a)
                
        return State(new_amplitudes, new_states)
    
    def __sub__(self, other):
        return self.__add__((-1) * other)
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            if len(self.states) != len(other.states):
                return False
            else:
                for s, a in zip(self.states, list(self.amplitudes)):
                    if s in other.states:
                        ind = other.states.index(s)
                        if other.amplitudes[ind] != a:
                            return False
                    else:
                        return False
                return True
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

In [7]:
a = hunds_case_a_basis(T='H',j=1,m=1,w=1)
b = hunds_case_a_basis(T='H',j=2,m=1,w=1)
c = hunds_case_a_basis(T='H',j=2,m=1,w=1)

In [ ]:
basis = hunds_case_a_basis(T=['H', 'C'], j=[1, 2], w=[-1, 1])

In [ ]:
states = [a, b, c]
A = states[0] + (1 + 1j) * states[1]
B = states[0] + 1j * states[1]

#A.normalize()
#B.normalize()

print(A * B)
print(B * A)

In [ ]:
me.lab_to_mol?

In [2]:
me.lab_to_mol(1, 0, 1, 
              1,-1, 0,
              1, 1, 1)/np.sqrt(2)

0.353553390593274

In [ ]:
(me.lab_to_mol(1, 0, 1, 
              1, 1, 0,
              1, 1, 1)
-
me.lab_to_mol(1, 0, 1, 
              1, 1, 0,
              1, 1, 1))